In [23]:
import requests
import json
import pandas as pd
import numpy as np

In [24]:
# get list posisi
posisi = requests.get('http://127.0.0.1:4000/posisi').json()
posisi_array = [item['nama'] for item in posisi['data']]

In [25]:
# get list kriteria
kriteria = requests.get('http://127.0.0.1:4000/kriteria').json()

In [26]:
kriteria['data']

[{'id': 1,
  'nama': 'ipk',
  'tipe': 'benefit',
  'kepentingan': 'penting',
  'created_at': None,
  'updated_at': None},
 {'id': 2,
  'nama': 'disiplin',
  'tipe': 'benefit',
  'kepentingan': 'penting',
  'created_at': None,
  'updated_at': None},
 {'id': 4,
  'nama': 'cv',
  'tipe': 'benefit',
  'kepentingan': 'sangat penting',
  'created_at': None,
  'updated_at': None},
 {'id': 11,
  'nama': 'atitude',
  'tipe': 'benefit',
  'kepentingan': 'penting',
  'created_at': None,
  'updated_at': None},
 {'id': 12,
  'nama': 'komitmen',
  'tipe': 'benefit',
  'kepentingan': 'menengah',
  'created_at': None,
  'updated_at': None},
 {'id': 13,
  'nama': 'test',
  'tipe': 'cost',
  'kepentingan': 'menengah',
  'created_at': None,
  'updated_at': None}]

In [27]:
# add kecocokan_jurusan criteria
kecocokan_jurusan = {
    'id': 99,
    'nama': 'kecocokan_jurusan',
    'tipe': 'benefit',
    'kepentingan': 'sangat penting',
    'created_at': None,
    'updated_at': None
}
kriteria['data'].append(kecocokan_jurusan)
crit = kriteria['data']

In [44]:
crit

[{'id': 1,
  'nama': 'ipk',
  'tipe': 'benefit',
  'kepentingan': 'penting',
  'created_at': None,
  'updated_at': None},
 {'id': 2,
  'nama': 'disiplin',
  'tipe': 'benefit',
  'kepentingan': 'penting',
  'created_at': None,
  'updated_at': None},
 {'id': 4,
  'nama': 'cv',
  'tipe': 'benefit',
  'kepentingan': 'sangat penting',
  'created_at': None,
  'updated_at': None},
 {'id': 11,
  'nama': 'atitude',
  'tipe': 'benefit',
  'kepentingan': 'penting',
  'created_at': None,
  'updated_at': None},
 {'id': 12,
  'nama': 'komitmen',
  'tipe': 'benefit',
  'kepentingan': 'menengah',
  'created_at': None,
  'updated_at': None},
 {'id': 13,
  'nama': 'test',
  'tipe': 'cost',
  'kepentingan': 'menengah',
  'created_at': None,
  'updated_at': None},
 {'id': 99,
  'nama': 'kecocokan_jurusan',
  'tipe': 'benefit',
  'kepentingan': 'sangat penting',
  'created_at': None,
  'updated_at': None}]

In [28]:
#df kriteria
crit_df = pd.DataFrame(crit)
crit_df = crit_df.drop(['id', 'tipe', 'created_at', 'updated_at'], axis=1)

In [29]:
# get data alternatif
response=requests.get('http://127.0.0.1:4000/hasil/alternatif').json()
df=pd.DataFrame(response['data'])

# split
df = df.applymap(lambda x: x.split(',') if isinstance(x, str) else x)
column_headers = list(df.columns.values)
for i in range(len(column_headers)) :
    df = df.explode(column_headers[i]).reset_index(drop=True)


In [30]:
# get data nilai jurusan
response_2=requests.get('http://127.0.0.1:4000/hasil/jurusan').json()
df_2=pd.DataFrame(response_2['data'])
df_2 = df_2.drop(['id'], axis=1)

melted_df = pd.melt(df_2, id_vars='nama_jurusan', var_name='posisi', value_name='kecocokan_jurusan')

In [31]:
merged_df = pd.merge(df, melted_df, on=['nama_jurusan', 'posisi'], how='left')

In [32]:
spk_df = merged_df.drop(['nama', 'nama_jurusan', 'posisi'], axis=1)

In [33]:
# normalisasi kriteria
for column in spk_df.columns: 
    for index, kriteria in enumerate(crit):
        # display(crit[index]['tipe'])
        if (column == crit[index]['nama']):
            if (crit[index]['tipe'] == 'benefit'):
                spk_df[column] = spk_df[column]  / spk_df[column].abs().max()
            else:
                spk_df[column] = spk_df[column]  / spk_df[column].abs().min()

In [45]:
spk_df

,disiplin,atitude,komitmen,cv,ipk,test,kecocokan_jurusan
0,0.954023,1.000000,0.555556,0.720430,1.000000,1.0,0.6
1,0.954023,1.000000,0.555556,0.720430,1.000000,1.0,0.2
2,0.954023,1.000000,0.555556,0.720430,1.000000,1.0,0.4
3,0.655172,1.000000,0.866667,0.827957,0.924870,NaN,1.0
4,0.655172,1.000000,0.866667,0.827957,0.924870,NaN,0.2
5,0.655172,1.000000,0.866667,0.827957,0.924870,NaN,0.6
6,0.931034,1.000000,1.000000,0.806452,0.922280,NaN,1.0
7,0.931034,1.000000,1.000000,0.806452,0.922280,NaN,0.8
8,0.931034,1.000000,1.000000,0.806452,0.922280,NaN,0.8
9,0.942529,0.730337,0.933333,1.000000,0.906736,NaN,1.0


In [34]:
#replace tingkat kepentingan
dict = {"sangat kurang penting" : '1', "kurang penting" : '2', "menengah": '3', "penting" : '4', "sangat penting": '5'}
crit_df = crit_df.replace({'kepentingan': dict})

In [35]:
#bobot saw
# crit_df['kepentingan'] = crit_df['kepentingan'].astype(float)

In [36]:
#perbaikan bobot mod-saw
crit_df['kepentingan'] = crit_df['kepentingan'].astype(float)
total_kepentingan = crit_df['kepentingan'].sum()
crit_df['kepentingan'] = crit_df['kepentingan'] / total_kepentingan

In [37]:
#transpose df
crit_df = crit_df.set_index('nama').T

In [38]:
# vektor S
common_columns = []
for column in spk_df.columns: 
    common_columns.append(column)

spk_df = spk_df[common_columns]
crit_df = crit_df[common_columns]
df_result = spk_df ** crit_df.iloc[0]

result_value = df_result.prod(axis=1)

In [39]:
#pemeringkatan
result_value.sum()

12.01966212743978

In [40]:
result_spk = merged_df[['nama', 'nama_jurusan', 'posisi']].copy()
result_spk = result_spk.assign(s_vector=result_value)
result_spk = result_spk.assign(skor=result_value/result_value.sum())
result_spk = result_spk.sort_values(by=['skor'], ascending=False)

In [41]:
# for item in posisi_array:
#     if(item == "kurator"):
#         result_spk = result_spk.drop(result_spk[result_spk['posisi'] == item].index)
    # else:
    #     result_spk = result_spk.drop_duplicates(subset=["nama"])

In [42]:
result_spk = result_spk.drop_duplicates(subset=["nama"])

In [43]:
result_spk

,nama,nama_jurusan,posisi,s_vector,skor
6,SUGIHARTO,pengelolaan arsip dan rekaman informasi,kurator,0.941596,0.078338
9,RAHMADIA ANISA PUTRI,teknologi rekayasa multimedia grafis,branding museum,0.927997,0.077207
3,Kania Nurfitri Azzahra,arkeologi,kurator,0.886379,0.073744
12,Naufal Aji Satria,administrasi publik,edukator,0.819109,0.068147
0,Urmala Pingky Diavirma,ilmu administrasi negara,edukator,0.802940,0.066802
